In [ ]:
# Pipeline final adaptado a tus 7 archivos (Colab / Jupyter)
# - No borra filas con NaN en 'value'
# - Usa name-of-sheet como hint de estación cuando no detecta estación en headers
# - Ignora columnas tipo "<estacion>_b"
# - Normaliza abreviaturas (SE->SURESTE, CE->CENTRO, etc.)
# - Guarda parquet y pivot excel al final

import pandas as pd
import numpy as np
import re
from pathlib import Path
import os

# ------------------
# Mapeos y canonicals
# ------------------
sheet_to_station = {
    "SE": "SURESTE",
    "CE": "CENTRO",
    "SO": "SUROESTE",
    "NE2": "NOROESTE2",
    "SE2": "SURESTE2",
    "SE3": "SURESTE3",
    "NE": "NORESTE",
    "NO": "NOROESTE",
    "NO2": "NOROESTE2",
    "NTE": "NORTE",
    "NTE2": "NORTE2",
    "SO2": "SUROESTE2",
    "SUR": "SUR",
    "NO3": "NOROESTE3",
    "NE3": "NORESTE3"
}
sheet_to_station = {k.strip().upper(): v for k,v in sheet_to_station.items()}

ESTACIONES_CANON = [
    'SURESTE','NORESTE','CENTRO','NOROESTE','SUROESTE','NOROESTE2',
    'NORTE','NORESTE2','SURESTE2','SUROESTE2','SURESTE3','SUR','NORTE2',
    'NORESTE3','NOROESTE3'
]

CONTAMINANTES_CANON = ['CO','NO','NO2','NOX','O3','PM10','PM2.5','PRS','RAINF','RH','SO2','SR','TOUT','WSR','WDR']

# ------------------
# Utilidades generales
# ------------------
def hacer_unicos(cols):
    vistos = {}
    nuevas = []
    for c in cols:
        c = str(c)
        if c not in vistos:
            vistos[c] = 0
            nuevas.append(c)
        else:
            vistos[c] += 1
            nuevas.append(f"{c}_{vistos[c]}")
    return nuevas

def norm_text(x):
    if pd.isna(x):
        return ""
    s = str(x).strip().upper()
    s = re.sub(r'\s+', ' ', s)
    s = s.replace("\u00A0"," ")
    return s

def best_match_token(token, candidates):
    if token is None:
        return None
    t = norm_text(token)
    if t == "":
        return None
    # exact
    for c in candidates:
        if t == norm_text(c):
            return c
    # substring
    for c in candidates:
        if norm_text(c) in t or t in norm_text(c):
            return c
    # token parts
    parts = re.split(r'[\s_\-\.]+', t)
    for p in parts:
        for c in candidates:
            if p == norm_text(c) or norm_text(c) in p:
                return c
    return None

# elimina columnas tipo "<estacion>_b" (ignorarlas)
def drop_b_columns(df):
    cols = [c for c in df.columns if not re.search(r'(_b$)|(\b_b\b)|(^.*_B$)', str(c), flags=re.IGNORECASE)]
    return df.loc[:, cols]

# ------------------
# Parsers por tipo de archivo (según tu descripción)
# ------------------

# 1) DATOS HISTÓRICOS 2020_Contaminante.xlsx
# - cada hoja = contaminante
# - primera col: (posiblemente) descriptor; segunda col: fecha/hora; luego estaciones (SE, NE, CE, NO, SO, NO2, Norte, NE2, SE2, SO2, SE3, Sur, NE3)
# - hay columnas de estación '_b' al lado que IGNORAR
def parse_2020_contaminante(path, sheet):
    # leemos con header=0
    df = pd.read_excel(path, sheet_name=sheet, header=0, engine='openpyxl')
    # quitar columnas *_b
    df = drop_b_columns(df)
    # detectar la columna de fecha: buscar columna cuyo header contenga "DATE" o "FECHA" o "TIME" o "HORA"
    cols_upper = [norm_text(c) for c in df.columns]
    date_col = None
    for c, cu in zip(df.columns, cols_upper):
        if any(k in cu for k in ['FECHA','DATE','HORA','TIME','TIMESTAMP','DATETIME']):
            date_col = c
            break
    # si no, considerar la segunda columna si la primera no parece fecha
    if date_col is None:
        if len(df.columns) >= 2:
            date_col = df.columns[1]
        else:
            date_col = df.columns[0]
    # convertir datetime
    df[date_col] = pd.to_datetime(df[date_col], errors='coerce', dayfirst=True)
    # las columnas estaciones son todas excepto la primera(s) (ignoramos la primera columna descriptiva)
    # suponer que columnas con headers que coincidan con abreviaturas SE, NE, CE, NO, SO, etc. son estaciones
    estacion_cols = []
    for c in df.columns:
        cu = norm_text(c)
        # ignorar date col
        if c == date_col:
            continue
        # if header indicates a station abbreviation
        if any(abbr in cu for abbr in sheet_to_station.keys()) or best_match_token(cu, ESTACIONES_CANON):
            estacion_cols.append(c)
    # si no detectamos estaciones, asumimos todas las columnas después de date_col son estaciones
    if not estacion_cols:
        after_idx = list(df.columns).index(date_col) + 1
        estacion_cols = list(df.columns)[after_idx:]
    # melt: cada columna -> station/pollutant (sheet name)
    rows = []
    pollutant_hint = norm_text(sheet)  # sheet name is pollutant in this file
    pollutant_guess = best_match_token(pollutant_hint, CONTAMINANTES_CANON) or pollutant_hint
    for col in estacion_cols:
        # station from column header (ex: 'SE' -> SURESTE)
        station_guess = sheet_to_station.get(norm_text(col), None) or best_match_token(col, ESTACIONES_CANON)
        temp = pd.DataFrame({
            'datetime': df[date_col].values,
            'station': [station_guess]*len(df),
            'pollutant': [pollutant_guess]*len(df),
            'value': df[col].values
        })
        temp['source_sheet'] = sheet
        temp['origin_col'] = col
        rows.append(temp)
    if not rows:
        return pd.DataFrame(columns=['datetime','station','pollutant','value','source_sheet','origin_col'])
    return pd.concat(rows, ignore_index=True)

# 2) DATOS HISTÓRICOS 2020_2021_TODAS ESTACIONES.xlsx
# - cada hoja = estacion, primera columna = fecha/hora, columnas = contaminantes
def parse_2020_2021_todas(path, sheet):
    df = pd.read_excel(path, sheet_name=sheet, header=0, engine='openpyxl')
    df = drop_b_columns(df)
    # primera columna = fecha
    first = df.columns[0]
    df = df.rename(columns={first: 'datetime'})
    df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce', dayfirst=True)
    # columnas contaminantes: todas menos datetime
    pollutant_cols = [c for c in df.columns if c != 'datetime']
    rows = []
    for col in pollutant_cols:
        pollutant_guess = best_match_token(col, CONTAMINANTES_CANON)
        temp = pd.DataFrame({
            'datetime': df['datetime'].values,
            'station': [sheet_to_station.get(sheet.strip().upper(), sheet.strip().upper())]*len(df),
            'pollutant': [pollutant_guess]*len(df),
            'value': df[col].values
        })
        temp['source_sheet'] = sheet
        temp['origin_col'] = col
        rows.append(temp)
    if not rows:
        return pd.DataFrame(columns=['datetime','station','pollutant','value','source_sheet','origin_col'])
    return pd.concat(rows, ignore_index=True)

# 3) DATOS HISTÓRICOS 2021_Contaminantes
# - todo en una hoja; primer renglón = estación, segundo = contaminante; primera columna fecha, segunda hora
def parse_2021_contaminantes(path, sheet):
    df = pd.read_excel(path, sheet_name=sheet, header=None, engine='openpyxl')
    # assume row0 = station headers, row1 = pollutant headers, data from row2
    header0 = df.iloc[0].astype(str).tolist()
    header1 = df.iloc[1].astype(str).tolist()
    # construct multiindex
    cols = []
    for a,b in zip(header0, header1):
        cols.append((norm_text(a), norm_text(b)))
    data = df.iloc[2:].copy().reset_index(drop=True)
    data.columns = pd.MultiIndex.from_tuples(cols)
    # combine first two columns into datetime if separated
    # find any column tuple that contains 'DATE' or 'FECHA' or 'HORA'
    # We'll attempt to get index values as datetime from the first column that converts well
    datetime_col = None
    for col in data.columns:
        a,b = col
        if any(k in a for k in ['FECHA','DATE','DATETIME']) or any(k in b for k in ['FECHA','DATE','DATETIME']):
            datetime_col = col
            break
    if datetime_col is None:
        # try first column
        datetime_col = data.columns[0]
    # rename datetime col to ('DATETIME','')
    data = data.rename(columns={datetime_col: ('DATETIME','')})
    try:
        data[('DATETIME','')] = pd.to_datetime(data[('DATETIME','')], errors='coerce', dayfirst=True)
    except Exception:
        data[('DATETIME','')] = pd.to_datetime(data[('DATETIME','')].astype(str), errors='coerce', dayfirst=True)
    # now melt using other columns
    rows = []
    for col in data.columns:
        if col == ('DATETIME',''):
            continue
        est_raw, poll_raw = col
        station_guess = best_match_token(est_raw, ESTACIONES_CANON) or sheet_to_station.get(est_raw)
        pollutant_guess = best_match_token(poll_raw, CONTAMINANTES_CANON)
        serie = data[col]
        temp = pd.DataFrame({
            'datetime': data[('DATETIME','')].values,
            'station': [station_guess]*len(serie),
            'pollutant': [pollutant_guess]*len(serie),
            'value': serie.values
        })
        temp['source_sheet'] = sheet
        temp['origin_col'] = f"{est_raw}__{poll_raw}"
        rows.append(temp)
    if not rows:
        return pd.DataFrame(columns=['datetime','station','pollutant','value','source_sheet','origin_col'])
    return pd.concat(rows, ignore_index=True)

# 4) DATOS HISTÓRICOS 2022_2023_TODAS ESTACIONES
# - cada hoja = estacion; primera columna = fecha y hora; columnas = contaminantes
def parse_2022_2023_todas(path, sheet):
    # same logic as 2020_2021_todas
    return parse_2020_2021_todas(path, sheet)

# 5) DATOS HISTÓRICOS 2023_2024_TODAS ESTACIONES_ITESM
# - todo en una hoja; first row = station, second = pollutant, third = metric; first col = date, second = time
def parse_2023_2024_itesm(path, sheet):
    df = pd.read_excel(path, sheet_name=sheet, header=None, engine='openpyxl')
    # header rows 0,1,2 -> use 0 and 1 for station/pollutant
    header0 = df.iloc[0].astype(str).tolist()
    header1 = df.iloc[1].astype(str).tolist()
    header2 = df.iloc[2].astype(str).tolist()  # metrics (ignore)
    cols = []
    for a,b in zip(header0, header1):
        cols.append((norm_text(a), norm_text(b)))
    data = df.iloc[3:].copy().reset_index(drop=True)
    data.columns = pd.MultiIndex.from_tuples(cols)
    # detect datetime column similar to parse_2021
    datetime_col = None
    for col in data.columns:
        a,b = col
        if any(k in a for k in ['FECHA','DATE','DATETIME']) or any(k in b for k in ['FECHA','DATE','DATETIME']):
            datetime_col = col
            break
    if datetime_col is None:
        datetime_col = data.columns[0]
    data = data.rename(columns={datetime_col: ('DATETIME','')})
    try:
        data[('DATETIME','')] = pd.to_datetime(data[('DATETIME','')], errors='coerce', dayfirst=True)
    except Exception:
        data[('DATETIME','')] = pd.to_datetime(data[('DATETIME','')].astype(str), errors='coerce', dayfirst=True)
    rows = []
    for col in data.columns:
        if col == ('DATETIME',''):
            continue
        est_raw, poll_raw = col
        station_guess = best_match_token(est_raw, ESTACIONES_CANON) or sheet_to_station.get(norm_text(est_raw))
        pollutant_guess = best_match_token(poll_raw, CONTAMINANTES_CANON)
        serie = data[col]
        temp = pd.DataFrame({
            'datetime': data[('DATETIME','')].values,
            'station': [station_guess]*len(serie),
            'pollutant': [pollutant_guess]*len(serie),
            'value': serie.values
        })
        temp['source_sheet'] = sheet
        temp['origin_col'] = f"{est_raw}__{poll_raw}"
        rows.append(temp)
    if not rows:
        return pd.DataFrame(columns=['datetime','station','pollutant','value','source_sheet','origin_col'])
    return pd.concat(rows, ignore_index=True)

# 6) DATOS HISTÓRICOS 2024_TODAS ESTACIONES
# - cada hoja = estación con abbreviations in sheet name (SE, NE, CE, etc)
# - first column = date/time, first row = contaminant (units in parens ignore)
def parse_2024_todas(path, sheet):
    df = pd.read_excel(path, sheet_name=sheet, header=0, engine='openpyxl')
    df = drop_b_columns(df)
    # header includes contaminant names maybe with units "(ug/m3)" -> remove parenthesis
    clean_cols = []
    for c in df.columns:
        cc = re.sub(r'\(.*\)', '', str(c))
        clean_cols.append(cc)
    df.columns = clean_cols
    first = df.columns[0]
    df = df.rename(columns={first: 'datetime'})
    df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce', dayfirst=True)
    pollutant_cols = [c for c in df.columns if c != 'datetime']
    rows = []
    # station from sheet name
    station_name = sheet_to_station.get(sheet.strip().upper(), sheet.strip().upper())
    for col in pollutant_cols:
        pollutant_guess = best_match_token(col, CONTAMINANTES_CANON)
        temp = pd.DataFrame({
            'datetime': df['datetime'].values,
            'station': [station_name]*len(df),
            'pollutant': [pollutant_guess]*len(df),
            'value': df[col].values
        })
        temp['source_sheet'] = sheet
        temp['origin_col'] = col
        rows.append(temp)
    if not rows:
        return pd.DataFrame(columns=['datetime','station','pollutant','value','source_sheet','origin_col'])
    return pd.concat(rows, ignore_index=True)

# 7) DATOS HISTÓRICOS 2025_TODAS ESTACIONES
# - same as 2024 but second row is metric (ignore)
def parse_2025_todas(path, sheet):
    df = pd.read_excel(path, sheet_name=sheet, header=0, engine='openpyxl')
    df = drop_b_columns(df)
    # if second row is metric, sometimes pandas read it as data. If the second row contains only units, drop it by checking datatypes.
    # If the second row values are non-numeric for most pollutant columns, drop that row before melting.
    # Let's inspect second row: if >50% of pollutant columns are non-numeric, drop row 0 (units)
    # We will create a copy and attempt to coerce numeric on row 0 for pollutant columns
    pollutant_cols = [c for c in df.columns[1:]] if len(df.columns) > 1 else []
    if pollutant_cols:
        nonnum_count = 0
        for c in pollutant_cols:
            try:
                float(str(df.iloc[0][c]))
            except Exception:
                nonnum_count += 1
        if nonnum_count > 0.5 * len(pollutant_cols):
            # drop first row (units row)
            df = df.iloc[1:].reset_index(drop=True)
    # now same logic as parse_2024
    clean_cols = []
    for c in df.columns:
        cc = re.sub(r'\(.*\)', '', str(c))
        clean_cols.append(cc)
    df.columns = clean_cols
    first = df.columns[0]
    df = df.rename(columns={first: 'datetime'})
    df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce', dayfirst=True)
    station_name = sheet_to_station.get(sheet.strip().upper(), sheet.strip().upper())
    rows = []
    pollutant_cols = [c for c in df.columns if c != 'datetime']
    for col in pollutant_cols:
        pollutant_guess = best_match_token(col, CONTAMINANTES_CANON)
        temp = pd.DataFrame({
            'datetime': df['datetime'].values,
            'station': [station_name]*len(df),
            'pollutant': [pollutant_guess]*len(df),
            'value': df[col].values
        })
        temp['source_sheet'] = sheet
        temp['origin_col'] = col
        rows.append(temp)
    if not rows:
        return pd.DataFrame(columns=['datetime','station','pollutant','value','source_sheet','origin_col'])
    return pd.concat(rows, ignore_index=True)

# ------------------
# Dispatcher que detecta según el nombre del archivo qué parser aplicar
# ------------------
def procesar_hoja_por_archivo(path, sheet, filename):
    fname = Path(filename).name.upper()
    if '2020_CONTAMINANTE' in fname:
        return parse_2020_contaminante(path, sheet)
    if '2020_2021' in fname:
        return parse_2020_2021_todas(path, sheet)
    if '2021_CONTAMINANTE' in fname:
        # this file had a single sheet with double header
        return parse_2021_contaminantes(path, sheet)
    if '2022_2023' in fname:
        return parse_2022_2023_todas(path, sheet)
    if '2023_2024' in fname or 'ITESM' in fname:
        return parse_2023_2024_itesm(path, sheet)
    if '2024_TODAS' in fname:
        return parse_2024_todas(path, sheet)
    if '2025_TODAS' in fname:
        return parse_2025_todas(path, sheet)
    # fallback: try generic wide parse
    try:
        df0 = pd.read_excel(path, sheet_name=sheet, header=0, engine='openpyxl')
        return parse_wide_sheet_from_df(df0, sheet)
    except Exception:
        return pd.DataFrame()

# ------------------
# Build master function
# ------------------
def build_master_from_files(archivos, save_parquet=True, out_prefix="master_unificado_mapped"):
    partes = []
    for archivo in archivos:
        archivo_path = str(archivo)
        print(f"\nProcesando archivo: {archivo_path}")
        if not os.path.exists(archivo_path):
            print("  -> Archivo no encontrado, lo salto.")
            continue
        try:
            xls = pd.ExcelFile(archivo_path, engine='openpyxl')
        except Exception as e:
            print("  -> ERROR leyendo archivo:", e)
            continue
        for hoja in xls.sheet_names:
            print(f"  Hoja: {hoja}")
            try:
                parsed = procesar_hoja_por_archivo(archivo_path, hoja, archivo_path)
                if parsed is None or parsed.empty:
                    print("    -> No se extrajeron datos de esta hoja.")
                    continue
                # Convert types
                parsed['datetime'] = pd.to_datetime(parsed['datetime'], errors='coerce')
                parsed['value'] = pd.to_numeric(parsed['value'], errors='coerce')
                parsed['source_file'] = Path(archivo_path).name
                parsed['source_sheet'] = hoja
                # if station is missing, try to use sheet name hint (applies for many cases)
                parsed['station'] = parsed['station'].fillna(parsed['source_sheet'].apply(lambda s: sheet_to_station.get(s.strip().upper())))
                # keep rows (including NaN values in 'value') as you requested
                partes.append(parsed)
                print(f"    -> filas extraídas: {len(parsed)}")
            except Exception as e:
                print("    -> ERROR procesando hoja:", e)
                continue
    if not partes:
        print("No se extrajo ninguna parte.")
        return pd.DataFrame()
    master = pd.concat(partes, ignore_index=True)
    # Map stations more robustly (normalize abbreviations and tokens)
    def map_station(x):
        if pd.isna(x):
            return None
        s = str(x).strip()
        su = s.upper()
        # direct mapping from sheet_to_station keys
        if su in sheet_to_station:
            return sheet_to_station[su]
        # direct canonical match or token match
        bm = best_match_token(su, ESTACIONES_CANON)
        if bm:
            return bm
        # remove trailing digits / punctuation and try again
        s2 = re.sub(r'[\d\._\-]','', su).strip()
        bm2 = best_match_token(s2, ESTACIONES_CANON)
        if bm2:
            return bm2
        return s  # fallback: keep original textual value
    master['station_original'] = master['station']
    master['station'] = master['station'].apply(map_station)
    # map pollutant tokens
    master['pollutant_original'] = master['pollutant']
    master['pollutant'] = master['pollutant'].apply(lambda x: best_match_token(x, CONTAMINANTES_CANON) if pd.notna(x) else x)
    # remove completely empty rows
    master = master[~(master['station'].isna() & master['pollutant'].isna() & master['value'].isna() & master['datetime'].isna())]
    # drop exact duplicate rows if any (keep last)
    master = master.drop_duplicates(subset=['datetime','station','pollutant','value','source_file','source_sheet'], keep='last')
    master = master.sort_values(['datetime','station','pollutant']).reset_index(drop=True)
    # Save
    if save_parquet:
        outp = f"/content/{out_prefix}.parquet"
        try:
            master.to_parquet(outp, index=False)
            print(f"\nMaster guardado en: {outp}")
        except Exception as e:
            print("No se pudo guardar parquet:", e)
    return master

def pivot_to_wide(master_df, fillna=False, out_path="/content/master_unificado_mapped_wide.xlsx"):
    df = master_df.copy()
    df = df.dropna(subset=['datetime'], how='all')
    df['colname'] = df['station'].astype(str) + "_" + df['pollutant'].astype(str)
    wide = df.pivot_table(index='datetime', columns='colname', values='value', aggfunc='first')
    wide = wide.sort_index(axis=1)
    if fillna is not False:
        wide = wide.fillna(fillna)
    try:
        wide.to_excel(out_path)
        print(f"Versión ancha guardada en: {out_path}")
    except Exception as e:
        print("No se pudo guardar versión ancha:", e)
    return wide

# ------------------
# EJECUCIÓN - pega los paths exactos de tus 7 archivos
# ------------------
if __name__ == "__main__":
    archivos = [
        # actualiza rutas si trabajas en Colab (monta drive y usa /content/drive/MyDrive/...)
        "/content/DATOS HISTÓRICOS 2020_Contaminante.xlsx",
        "/content/DATOS HISTÓRICOS 2020_2021_TODAS ESTACIONES.xlsx",
        "/content/DATOS HISTÓRICOS 2021_Contaminante.xlsx",
        "/content/DATOS HISTÓRICOS 2022_2023_TODAS ESTACIONES.xlsx",
        "/content/DATOS HISTÓRICOS 2023_2024_TODAS ESTACIONES_ITESM.xlsx",  # el que subiste
        "/content/DATOS HISTÓRICOS 2024_TODAS ESTACIONES.xlsx",
        "/content/DATOS HISTÓRICOS 2025_TODAS ESTACIONES.xlsx"
    ]

    master = build_master_from_files(archivos, save_parquet=True, out_prefix="master_unificado_mapped")
    print("\nMaster final - primeras filas:")
    print(master.head(10))

    # crear versión ancha y guardar
    wide = pivot_to_wide(master, fillna=False, out_path="/content/master_unificado_mapped_wide.xlsx")
    print("Listo. Revisa los archivos en /content/ o en la ruta que hayas elegido para guardar.")



Procesando archivo: /content/DATOS HISTÓRICOS 2020_Contaminante.xlsx
  Hoja: PM10
    -> filas extraídas: 122850
  Hoja: PM2.5
    -> filas extraídas: 122066
  Hoja: O3
    -> filas extraídas: 122850
  Hoja: NO
    -> filas extraídas: 122850
  Hoja: NO2
    -> filas extraídas: 122836
  Hoja: NOx
    -> filas extraídas: 122850
  Hoja: CO
    -> filas extraídas: 122668
  Hoja: SO2
    -> filas extraídas: 122836

Procesando archivo: /content/DATOS HISTÓRICOS 2020_2021_TODAS ESTACIONES.xlsx
  Hoja: SURESTE
    -> filas extraídas: 263070
  Hoja: NORESTE
    -> filas extraídas: 263025
  Hoja: CENTRO
    -> filas extraídas: 263040
  Hoja: NOROESTE
    -> filas extraídas: 263055
  Hoja: SUROESTE
    -> filas extraídas: 263055
  Hoja: NOROESTE2
    -> filas extraídas: 263025
  Hoja: NORTE
    -> filas extraídas: 263040
  Hoja: SUROESTE2
    -> filas extraídas: 263025
  Hoja: SURESTE2
    -> filas extraídas: 263040
  Hoja: SURESTE3
    -> filas extraídas: 263025
  Hoja: SUR
    -> filas extra

/tmp/ipython-input-1929529637.py:166: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce', dayfirst=True)


    -> filas extraídas: 30

Procesando archivo: /content/DATOS HISTÓRICOS 2021_Contaminante.xlsx
  Hoja: Param_horarios_Estaciones


/tmp/ipython-input-1929529637.py:213: PerformanceWarning: indexing past lexsort depth may impact performance.
  data[('DATETIME','')] = pd.to_datetime(data[('DATETIME','')], errors='coerce', dayfirst=True)
/tmp/ipython-input-1929529637.py:215: PerformanceWarning: indexing past lexsort depth may impact performance.
  data[('DATETIME','')] = pd.to_datetime(data[('DATETIME','')].astype(str), errors='coerce', dayfirst=True)


    -> ERROR procesando hoja: 'DATETIME'

Procesando archivo: /content/DATOS HISTÓRICOS 2022_2023_TODAS ESTACIONES.xlsx
  Hoja: SURESTE
    -> filas extraídas: 213825
  Hoja: NORESTE
    -> filas extraídas: 213825
  Hoja: CENTRO
    -> filas extraídas: 213825
  Hoja: NOROESTE
    -> filas extraídas: 213825
  Hoja: SUROESTE
    -> filas extraídas: 213825
  Hoja: NOROESTE2
    -> filas extraídas: 213825
  Hoja: NORTE
    -> filas extraídas: 213825
  Hoja: SUROESTE2
    -> filas extraídas: 213825
  Hoja: SURESTE2
    -> filas extraídas: 213825
  Hoja: SURESTE3
    -> filas extraídas: 213825
  Hoja: SUR
    -> filas extraídas: 213810
  Hoja: NORTE2
    -> filas extraídas: 213825
  Hoja: NORESTE2
    -> filas extraídas: 213825
  Hoja: NORESTE3
    -> filas extraídas: 213810
  Hoja: NOROESTE3
    -> filas extraídas: 93555
  Hoja: CATÁLOGO
    -> filas extraídas: 30

Procesando archivo: /content/DATOS HISTÓRICOS 2023_2024_TODAS ESTACIONES_ITESM.xlsx
  Hoja: Param_horarios_Estaciones


/tmp/ipython-input-1929529637.py:166: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce', dayfirst=True)
/tmp/ipython-input-1929529637.py:268: PerformanceWarning: indexing past lexsort depth may impact performance.
  data[('DATETIME','')] = pd.to_datetime(data[('DATETIME','')], errors='coerce', dayfirst=True)
/tmp/ipython-input-1929529637.py:270: PerformanceWarning: indexing past lexsort depth may impact performance.
  data[('DATETIME','')] = pd.to_datetime(data[('DATETIME','')].astype(str), errors='coerce', dayfirst=True)


    -> ERROR procesando hoja: 'DATETIME'
  Hoja: Hoja2
    -> ERROR procesando hoja: single positional indexer is out-of-bounds

Procesando archivo: /content/DATOS HISTÓRICOS 2024_TODAS ESTACIONES.xlsx
  Hoja: SE
    -> filas extraídas: 131760
  Hoja: CE
    -> filas extraídas: 131760
  Hoja: SO
    -> filas extraídas: 131760
  Hoja: NE2
    -> filas extraídas: 131730
  Hoja: SE2
    -> filas extraídas: 131730
  Hoja: SE3
    -> filas extraídas: 131730
  Hoja: NE
    -> filas extraídas: 131760
  Hoja: NO
    -> filas extraídas: 131745
  Hoja: NO2
    -> filas extraídas: 131730
  Hoja: NTE
    -> filas extraídas: 131730
  Hoja: NTE2
    -> filas extraídas: 131730
  Hoja: SO2
    -> filas extraídas: 131730
  Hoja: SUR
    -> filas extraídas: 131730
  Hoja: NO3
    -> filas extraídas: 131760
  Hoja: NE3
    -> filas extraídas: 131730

Procesando archivo: /content/DATOS HISTÓRICOS 2025_TODAS ESTACIONES.xlsx
  Hoja: SE
    -> filas extraídas: 65145
  Hoja: CE
    -> filas extraídas: 65160